a) A prefix code is a type of code system distinguished by its possession of the "prefix property", which requires that there is no whole code word in the system that is a prefix (initial segment) of any other code word in the system. It is trivially true for fixed-length code, so only a point of consideration in variable-length code.

b) Kraft's inequality limits the lengths of codewords in a prefix code: if one takes an exponential of the length of each valid codeword, the resulting set of values must look like a probability mass function, that is, it must have total measure less than or equal to one.

c) In information theory, Shannon's source coding theorem (or noiseless coding theorem) establishes the limits to possible data compression, and the operational meaning of the Shannon entropy.

The source coding theorem shows that (in the limit, as the length of a stream of independent and identically-distributed random variable (i.i.d.) data tends to infinity) it is impossible to compress the data such that the code rate (average number of bits per symbol) is less than the Shannon entropy of the source, without it being virtually certain that information will be lost. However it is possible to get the code rate arbitrarily close to the Shannon entropy, with negligible probability of loss.

In [1]:
from math import log,log2
import sympy as sp
import operator

class DDS:
    def __init__(self,symbols):
        if isinstance(symbols,dict):
            assert sum(symbols.values())==1
            self.symbols=symbols
        else:
            assert round(float(sum(symbols)),3)==1.0
            self.symbols=dict((str(i),symbols[i]) for i in range(len(symbols)))
        self.n=len(symbols)
        #Hartley formula
        self.max_entropy=log2(self.n)
        
    def get_entropy(self):
        return sum(-x*log2(x) for x in self.symbols.values())
    
    def information(self,symbol):
        return -log2(self.symbols[self.symbol])
    
    def redundancy(self):
        return 1-self.get_entropy()/self.max_entropy
    
class ChannelMatrix:
    def __init__(self,mat,p_x):
        self.mat=sp.Matrix(mat)
        self.p_x=sp.Array(p_x)
        
        M_YX=self.M_YX()
        self.p_y=[sum(M_YX[:,x]) for x in range(3)]
    
    def mean_information_per_symbol(self):
        return DDS(self.p_x).get_entropy()
    
    def M_YX(self):
        M_YX = self.mat.copy()
        for i in range(3):
            for j in range(3):
                M_YX[i,j]*=self.p_x[i]
        return M_YX
    
    def H_YX(self):
        m=self.M_YX()
        s=0
        for i in range(3):
            for j in range(3):
                s-=self.p_x[i]*self.mat[i,j]*log2(self.mat[i,j])
        return s
        
    
    def I_XY(self,message_length):
        n=message_length
        H_YX=self.H_YX()
        H_Y=DDS(self.p_x).get_entropy()
        return n*(DDS(c.p_y).get_entropy()-(H_Y-H_YX))
    

In [2]:
c = ChannelMatrix([
    [0.15,0.25,0.6],
    [0.35,0.25,0.4],
    [0.49,0.5,0.01]
],[0.2,0.2,0.6])

In [3]:
c.mean_information_per_symbol()

1.37095059445467

In [4]:
c.M_YX()

Matrix([
[ 0.03, 0.05,  0.12],
[ 0.07, 0.05,  0.08],
[0.294,  0.3, 0.006]])

In [5]:
sum(c.p_y)

1.00000000000000

In [6]:
c.p_y

[0.394000000000000, 0.400000000000000, 0.206000000000000]

In [7]:
DDS(c.p_x).get_entropy()

1.37095059445467

In [8]:
c.H_YX()

1.22475137157655

In [9]:
dds1=DDS([0.2,0.3,0.4,0.1])
dds1.redundancy()

0.07678032766449228

In [10]:
#Optimal codes:
# H_max = log2(n)
# let H - an entropy of a DDS X
# 1 - H/H_max - redundancy of the code X

class Node:
    def __init__(self, prob, symbol, left=None, right=None):
        self.prob = prob
        self.symbol = symbol
        self.left = left
        self.right = right
        self.code = ''

codes = dict()

def Calculate_Codes(node, val=''):
    newVal = val + str(node.code)

    if(node.left):
        Calculate_Codes(node.left, newVal)
    if(node.right):
        Calculate_Codes(node.right, newVal)

    if(not node.left and not node.right):
        codes[node.symbol] = newVal
         
    return codes        

def Calculate_Probability(data):
    symbols = dict()
    for element in data:
        if symbols.get(element) == None:
            symbols[element] = 1
        else: 
            symbols[element] += 1     
    return symbols

def Output_Encoded(data, coding):
    encoding_output = []
    for c in data:
        encoding_output.append(coding[c])
        
    string = ''.join([str(item) for item in encoding_output])    
    return string
        
def Total_Gain(data, coding):
    before_compression = len(data) * 8 # total bit space to stor the data before compression
    after_compression = 0
    symbols = coding.keys()
    for symbol in symbols:
        count = data.count(symbol)
        after_compression += count * len(coding[symbol]) #calculate how many bit is required for that symbol in total
    print("Space usage before compression (in bits):", before_compression)    
    print("Space usage after compression (in bits):",  after_compression)           

def Huffman_Encoding(data):
    symbol_with_probs = Calculate_Probability(data)
    symbols = symbol_with_probs.keys()
    probabilities = symbol_with_probs.values()
    print("symbols: ", symbols)
    print("probabilities: ", probabilities)
    
    nodes = []
    for symbol in symbols:
        nodes.append(Node(symbol_with_probs.get(symbol), symbol))
    
    while len(nodes) > 1:
        nodes = sorted(nodes, key=lambda x: x.prob)
        right = nodes[0]
        left = nodes[1]
    
        left.code = 0
        right.code = 1
    
        newNode = Node(left.prob+right.prob, left.symbol+right.symbol, left, right)
    
        nodes.remove(left)
        nodes.remove(right)
        nodes.append(newNode)
            
    huffman_encoding = Calculate_Codes(nodes[0])
    print("symbols with codes", huffman_encoding)
    Total_Gain(data, huffman_encoding)
    encoded_output = Output_Encoded(data,huffman_encoding)
    return encoded_output, nodes[0]  
    
def Huffman_Decoding(encoded_data, huffman_tree):
    tree_head = huffman_tree
    decoded_output = []
    for x in encoded_data:
        if x == '1':
            huffman_tree = huffman_tree.right   
        elif x == '0':
            huffman_tree = huffman_tree.left
        try:
            if huffman_tree.left.symbol == None and huffman_tree.right.symbol == None:
                pass
        except AttributeError:
            decoded_output.append(huffman_tree.symbol)
            huffman_tree = tree_head
        
    string = ''.join([str(item) for item in decoded_output])
    return string        


data = "AAAAAAABCCCCCCDDEEEEE"
print(data)
encoding, tree = Huffman_Encoding(data)
print("Encoded output", encoding)
print("Decoded Output", Huffman_Decoding(encoding,tree))

AAAAAAABCCCCCCDDEEEEE
symbols:  dict_keys(['A', 'B', 'C', 'D', 'E'])
probabilities:  dict_values([7, 1, 6, 2, 5])
symbols with codes {'A': '00', 'C': '01', 'E': '10', 'D': '110', 'B': '111'}
Space usage before compression (in bits): 168
Space usage after compression (in bits): 45
Encoded output 000000000000001110101010101011101101010101010
Decoded Output AAAAAAABCCCCCCDDEEEEE


In [11]:
dds2=DDS([
    0.46,0.3,0.12,0.06,0.03,0.02,0.01
])
h=Huffman(dds2)
DDS(h.optimize()).get_entropy()

NameError: name 'Huffman' is not defined

In [12]:
dds2.get_entropy()

1.9781083861545001

HW 3

In [13]:
dds3=DDS({
    "x1":0.2,
    "x2":0.2,
    "x3":0.4,
    "x4":0.15,
    "x5":0.05
})
huf=Huffman(dds3)
huf.optimize(include_syms=True)

NameError: name 'Huffman' is not defined

In [14]:
huf.R()

NameError: name 'huf' is not defined

In [15]:
dds3.get_entropy()

2.084183719779189

In [16]:
#Efficiency
huf.mu()

NameError: name 'huf' is not defined

In [17]:
DDS([
    0.01,0.04,0.05,0.04,0.16,0.2,0.05,0.2,0.25
]).get_entropy()/1.5

1.8146187299249081

In [18]:
6*0.01+6*0.04+5*0.05+5*0.04+3*0.16+2*0.2+5*0.05+2*0.2+0.25

2.53

In [19]:
1.815/2.53

0.7173913043478262

In [20]:
bin(1231)

'0b10011001111'

In [285]:
int("111111",4)

1365

In [277]:
bin(109)

'0b1101101'

In [25]:
s="кодуваннякодудекодування"
sym=dict()
for x in s:
    if x in sym:
        sym[x]+=1
    else:
        sym[x]=1

In [32]:
len(s)

24

In [27]:
sym

{'к': 3, 'о': 3, 'д': 4, 'у': 3, 'в': 2, 'а': 2, 'н': 4, 'я': 2, 'е': 1}

In [28]:
for x in sym:
    sym[x]/=len(s)



In [29]:
sorted([(x,sym[x]) for x in sym],key=operator.itemgetter(1),reverse=True)

[('д', 0.16666666666666666),
 ('н', 0.16666666666666666),
 ('к', 0.125),
 ('о', 0.125),
 ('у', 0.125),
 ('в', 0.08333333333333333),
 ('а', 0.08333333333333333),
 ('я', 0.08333333333333333),
 ('е', 0.041666666666666664)]

In [30]:
l=list(sym.values())

In [34]:
DDS(list(sym.values())).get_entropy()

3.073934896284056

In [42]:
n=0.4
-log2(n)

1.3219280948873622

In [312]:
DDS([
    0.3,0.1,0.25,0.15,0.2
]).get_entropy()

2.228212945841001

In [313]:
2**2.23

4.691339796927515

In [314]:
log2(5)

2.321928094887362

In [318]:
bin(5)

'0b101'

In [16]:
DDS([
    0.394,0.4,0.206
]).get_entropy()

1.5277342832866418